In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import cv2
import glob
from collections import defaultdict
import pandas as pd
from torchvision.utils import make_grid

In [3]:
figs_root = os.path.join("figs", "output_vis")
if not os.path.exists(figs_root):
  os.makedirs(figs_root)

In [6]:
!ls /hdd3/mliuzzolino/TDPoseEstimation

mpii_backup  mpii_backup_v2  results


In [29]:
base_dir = "mpii_backup"  # resulst/mpii, mpii_backup_v2

In [30]:
root = f"/hdd3/mliuzzolino/TDPoseEstimation/{base_dir}"
exp_paths = glob.glob(f"{root}/*")
exp_paths

['/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x4__TD_0.9__double',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.9__distill__TD_0.0',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_1.0__distill__TD_1.0',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.9',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.25__distill__TD_1.0',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.9__untied_weights__no_skip',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_1.0',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x4__TD_0.25__double',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.0__untied_weights__no_skip',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.5__untied_weights__no_skip',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_backup/hourglass_x8__TD_0.25__distill__TD_0.5',
 '/hdd3/mliuzzolino/TDPoseEstimation/mpii_back

In [31]:
df_dict = defaultdict(list)
break_it = False
for exp_i, exp_path in enumerate(exp_paths):
  sys.stdout.write(f"\rProcessing experiment {exp_i+1}/{len(exp_paths)}...")
  sys.stdout.flush()
  exp_name = os.path.basename(exp_path)
  exp_data_paths = np.sort(glob.glob(f"{exp_path}/*.pt"))
  for data_path in exp_data_paths:
    batch_name = os.path.splitext(os.path.basename(data_path))[0]
    loaded_data = torch.load(data_path)
    x_data = loaded_data["x_data"]
    target = loaded_data["target"]
    out_combos = loaded_data["predictions"]
    if "combo_joint_imgs" not in loaded_data:
      print(f"{exp_name} does not contain combo_joint_imgs key")
      break
    combo_joint_imgs = loaded_data["combo_joint_imgs"]
      
    grid_im = make_grid(out_combos, padding=15, pad_value=0.9).permute(1,2,0)
    
    df_dict["exp_name"].append(exp_name)
    df_dict["batch_name"].append(batch_name)
    df_dict["x_data"].append(x_data)
    df_dict["target"].append(target)
    df_dict["predictions"].append(grid_im)
    df_dict["combo_joint_imgs"].append(combo_joint_imgs)
    df_dict["distillation"].append("distill" in exp_name)
    df_dict["untied"].append("untied" in exp_name)
df = pd.DataFrame(df_dict)

Processing experiment 35/35...

In [32]:
def make_title(exp_name):
  if "untied" in exp_name:
    student_td = float(exp_name.split("__untied_weights__")[0].split("TD_")[1])
    new_title = f"TD({student_td}) [untied weights, no skip]"
  elif "__distill__" in exp_name:
    new_title = exp_name.split("__")[1]
    student_td = float(new_title.split("__distill__")[0].split("TD_")[1])
    teacher_td = float(exp_name.split("__distill__")[1].split("TD_")[1])
    new_title = f"Distillation: StudentTD({student_td}), TeacherTD({teacher_td})"
  else:
    student_td = exp_name.split("__")[1].split("TD_")[1]
    new_title = f"TD({student_td})"
  return new_title

In [33]:
batch_names = df.batch_name.unique()
rand_batch_name = np.random.choice(batch_names)
batch_df = df[df.batch_name==rand_batch_name]
rand_batch_name

AttributeError: 'DataFrame' object has no attribute 'batch_name'

In [34]:
x_data_1 = batch_df.iloc[0].x_data
x_data_2 = batch_df.iloc[1].x_data
assert (x_data_1 == x_data_2).all(), "Mismatched batches!"
x_data = batch_df.iloc[0].x_data
target = batch_df.iloc[0].target

fig, axes = plt.subplots(1,2, figsize=(6,5))
axes[0].imshow(x_data)
axes[1].imshow(target)
for ax in axes:
  ax.axis("off")
  
plt.tight_layout()
save_basename = f"{rand_batch_name}"
save_basename += "__xy"
save_path = os.path.join(figs_root, f"{save_basename}.png")
plt.savefig(save_path, dpi=300)

# Keypoint plot
pred_joint_imgs = batch_df.iloc[0].combo_joint_imgs
pred_joint_imgs = [torch.tensor(ele).permute(2,0,1) for ele in pred_joint_imgs]
grid_im = make_grid(pred_joint_imgs, padding=10, pad_value=240).permute(1,2,0)

plt.figure(figsize=(12,6))
plt.imshow(grid_im)
plt.axis("off")
save_basename = f"{rand_batch_name}"
save_basename += "__rgb_keypoint"
save_path = os.path.join(figs_root, f"{save_basename}.png")
plt.savefig(save_path, dpi=300)
plt.show()
plt.clf()

for distillation_key, dist_df in batch_df.groupby("distillation"):
  for untied_key, df_j in dist_df.groupby("untied"):
    exp_g = df_j.groupby("exp_name")
    for i, (exp_name, df_i) in enumerate(exp_g):
      fig = plt.figure(figsize=(12,6))
      title = make_title(exp_name)
      predictions = df_i.iloc[0].predictions
      plt.imshow(predictions)
      plt.title(title)
      plt.axis("off")
      plt.tight_layout()
      save_basename = f"{title}_{rand_batch_name}"
      if distillation_key:
        save_basename += "__distillation"
      if untied_key:
        save_basename += "__untied_key"
      save_path = os.path.join(figs_root, f"{save_basename}.png")
      plt.savefig(save_path, dpi=300)
      plt.show()
      plt.clf()

NameError: name 'batch_df' is not defined